# Law List Collection

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange

In [2]:
page = 1
display = 100

ID = ''
BASE = "https://www.law.go.kr/DRF/"

URL = f"{BASE}lawSearch.do?OC={ID}&target=law&type=XML&display={display}&page={page}"

In [73]:
response = urlopen(URL).read()
xtree = ET.fromstring(response)

In [74]:
totalCnt = int(xtree.find('totalCnt').text); totalCnt

5311

In [75]:
list(xtree)

[<Element 'target' at 0x1651bf9c0>,
 <Element '키워드' at 0x1651bfbf0>,
 <Element 'section' at 0x1651bfe70>,
 <Element 'totalCnt' at 0x1651bff10>,
 <Element 'page' at 0x1651bfab0>,
 <Element 'numOfRows' at 0x1651bff60>,
 <Element 'resultCode' at 0x1651bfb50>,
 <Element 'resultMsg' at 0x1651bf740>,
 <Element 'law' at 0x1651bf920>,
 <Element 'law' at 0x1651e44f0>,
 <Element 'law' at 0x1651e4ae0>,
 <Element 'law' at 0x1651e51c0>,
 <Element 'law' at 0x1651e5670>,
 <Element 'law' at 0x1651e5c60>,
 <Element 'law' at 0x1651e6250>,
 <Element 'law' at 0x1651e67a0>,
 <Element 'law' at 0x1651e6cf0>,
 <Element 'law' at 0x1651e71f0>,
 <Element 'law' at 0x1651e7b00>,
 <Element 'law' at 0x1651413f0>,
 <Element 'law' at 0x1651f7c40>,
 <Element 'law' at 0x1651f41d0>,
 <Element 'law' at 0x1651f4720>,
 <Element 'law' at 0x1651f4c70>,
 <Element 'law' at 0x1651f5210>,
 <Element 'law' at 0x1651f5760>,
 <Element 'law' at 0x1651f5cb0>,
 <Element 'law' at 0x1651f6200>,
 <Element 'law' at 0x1651f6750>,
 <Element '

In [81]:
page = 1
law_list = []

for i in trange(int(totalCnt / display) + 1):
    SEARCH_URL = f"{BASE}lawSearch.do?OC={ID}&target=law&type=XML&display={display}&page={page}"
    
    response = urlopen(SEARCH_URL).read()
    xtree = ET.fromstring(response)

    try:
        items = xtree[8:]
    except:
        break
    
    for node in items:
        법령일련번호 = node.find('법령일련번호').text
        현행연혁코드 = node.find('현행연혁코드').text
        법령명한글 = node.find('법령명한글').text
        법령약칭명 = node.find('법령약칭명').text
        법령ID = node.find('법령ID').text
        공포일자 = node.find('공포일자').text
        공포번호 = node.find('공포번호').text
        제개정구분명 = node.find('제개정구분명').text
        소관부처코드 = node.find('소관부처코드').text
        소관부처명 = node.find('소관부처명').text
        법령구분명 = node.find('법령구분명').text
        공동부령정보 = node.find('공동부령정보').text
        시행일자 = node.find('시행일자').text
        자법타법여부 = node.find('자법타법여부').text
        법령상세링크 = node.find('법령상세링크').text

        law_list.append(
            {
                '법령일련번호': 법령일련번호,
                '현행연혁코드': 현행연혁코드,
                '법령명한글': 법령명한글,
                '법령약칭명': 법령약칭명,
                '법령ID': 법령ID,
                '공포일자': 공포일자,
                '공포번호': 공포번호,
                '제개정구분명': 제개정구분명,
                '소관부처코드': 소관부처코드,
                '소관부처명': 소관부처명,
                '법령구분명': 법령구분명,
                '공동부령정보': 공동부령정보,
                '시행일자': 시행일자,
                '자법타법여부': 자법타법여부,
                '법령상세링크': 법령상세링크
            }
        )
    
    page += 1

100%|██████████| 54/54 [00:10<00:00,  4.99it/s]


In [82]:
law_df = pd.DataFrame(law_list)
law_df.to_csv('law.csv', index=False)

# Law Crawling

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import tqdm

In [2]:
law_df = pd.read_csv('law.csv')

In [3]:
ID = ''
SERVICE_BASE = "https://www.law.go.kr"

SERVICE_URL = SERVICE_BASE + law_df.loc[2]['법령상세링크'].replace('HTML', 'XML')

In [5]:
response = urlopen(SERVICE_URL).read()
xtree = ET.fromstring(response)

In [6]:
list(xtree)

[<Element '기본정보' at 0x15c2d7dd0>,
 <Element '조문' at 0x15c5211c0>,
 <Element '부칙' at 0x15c564360>,
 <Element '개정문' at 0x15c5652b0>,
 <Element '제개정이유' at 0x15c565350>]

In [7]:
def print_tags(xml_element, depth=0):
    # 현재 요소의 태그와 들여쓰기 출력
    indent = ' ' * (depth * 4)  # 들여쓰기는 깊이에 따라 4의 배수로 설정
    print(f"{indent}{xml_element.tag}")

    # 재귀적으로 모든 자식 요소에 대해 같은 함수 호출
    for child in xml_element:
        print_tags(child, depth + 1)

# 최상위 요소에서 시작하여 모든 태그 출력
print_tags(xtree)

법령
    기본정보
        법령ID
        공포일자
        공포번호
        언어
        법종구분
        법령명_한글
        법령명_한자
        법령명약칭
        제명변경여부
        한글법령여부
        편장절관
        소관부처
        전화번호
        시행일자
        제개정구분
        별표편집여부
        공포법령여부
        연락부서
            부서단위
                소관부처명
                소관부처코드
                부서명
                부서연락처
            부서단위
                소관부처명
                소관부처코드
                부서명
                부서연락처
            부서단위
                소관부처명
                소관부처코드
                부서명
                부서연락처
            부서단위
                소관부처명
                소관부처코드
                부서명
                부서연락처
            부서단위
                소관부처명
                소관부처코드
                부서명
                부서연락처
        공동부령정보
    조문
        조문단위
            조문번호
            조문여부
            조문시행일자
            조문이동이전
            조문이동이후
            조문변경여부
            조문내용
        조문단위
            조문번호
            조문여부
            조

In [8]:
default_fields = ["법령ID", "공포일자", "공포번호", "언어", "법종구분", "법종구분코드", "법령명_한글", "법령명_한자", "법령명약칭", "제명변경여부", "한글법령여부", "편장절관", "소관부처코드", "소관부처", "전화번호", "시행일자", "제개정구분", "별표편집여부", "공포법령여부", "공동부령정보"]
jomun_unit_fields = ["조문번호", "조문여부", "조문제목", "조문시행일자", "조문제개정유형", "조문이동이전", "조문이동이후", "조문변경여부", "조문내용"]
hang_unit_fields = ["항번호", "항제개정유형", "항내용"]
ho_unit_fields = ["호번호", "호내용"]
appendix_fields = ["부칙공포일자", "부칙공포번호", "부칙내용"]
fixed_fields = ["개정문내용"]
fixed_reason_fields = ["제개정이유내용"]

# "소관부처명", "소관부처코드", "부서명", "부서연락처", "별표번호", "별표가지번호", "별표구분", "별표제목", "별표시행일자", "별표서식파일링크", "별표 HWP 파일명", "별표서식PDF파일링크", "별표 PDF 파일명", "별표 이미지 파일명", "별표내용",

In [9]:
def remove_empty_arrays(data):
    if isinstance(data, dict):
        keys_to_delete = [key for key, value in data.items() if isinstance(value, list) and not value]
        for key in keys_to_delete:
            del data[key]
        
        # 재귀적으로 사전 내부 검사
        for key in list(data.keys()):
            remove_empty_arrays(data[key])
    elif isinstance(data, list):
        for item in data:
            remove_empty_arrays(item)
    
    return data

In [12]:
law_text_list = []

for i in tqdm(range(len(law_df))):  # 모든 법령을 반복 처리 
    SERVICE_URL = SERVICE_BASE + law_df.loc[i]['법령상세링크'].replace('HTML', 'XML')
    
    response = urlopen(SERVICE_URL).read()
    xtree = ET.fromstring(response)

    # XML 파일 저장
    with open(f'./law_crawling/law_text_{i}.xml', 'wb') as f:
        f.write(response)

    law_data = {}

    # 기본 정보
    for field in default_fields:
        try:
            law_data[field] = xtree.find("기본정보").find(field).text
        except:
            pass

    # 조문단위, 항, 호 처리
    law_data['조문'] = []
    for jomun in xtree.find("조문").findall("조문단위"):
        jomun_data = {field: jomun.find(field).text if jomun.find(field) is not None else None for field in jomun_unit_fields}
        jomun_data['항'] = []

        for hang in jomun.findall("항"):
            hang_data = {field: hang.find(field).text if hang.find(field) is not None else None for field in hang_unit_fields}
            hang_data['호'] = []

            for ho in hang.findall("호"):
                ho_data = {field: ho.find(field).text if ho.find(field) is not None else None for field in ho_unit_fields}
                hang_data['호'].append(ho_data)

            jomun_data['항'].append(hang_data)

        law_data['조문'].append(jomun_data)

    # 부칙 정보
    law_data['부칙'] = []
    try:
        for appendix in xtree.find("부칙").findall("부칙단위"):
            appendix_data = {field: appendix.find(field).text if appendix.find(field) is not None else None for field in appendix_fields}
            law_data['부칙'].append(appendix_data)
    except:
        pass

    # 개정문 내용
    try:
        law_data['개정문'] = xtree.find("개정문").find("개정문내용").text
    except:
        pass
    
    # 개정이유 내용
    try:
        law_data['제개정이유'] = xtree.find("제개정이유").find("제개정이유내용").text
    except:
        pass
    
    law_data = remove_empty_arrays(law_data)
    law_text_list.append(law_data)

100%|██████████| 5311/5311 [2:22:20<00:00,  1.61s/it]  


In [14]:
law_text_list_df = pd.DataFrame(law_text_list)
law_text_list_df.to_json('law_text.csv', index=False)

In [20]:
print(law_text_list[0])

{'법령ID': '010719', '공포일자': '20230808', '공포번호': '19592', '언어': '한글', '법종구분': '법률', '법령명_한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률', '법령명_한자': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률', '법령명약칭': '10ㆍ27법난법', '제명변경여부': 'N', '한글법령여부': 'Y', '편장절관': '13030000', '소관부처': '문화체육관광부', '전화번호': '044-203-2312', '시행일자': '20230808', '제개정구분': '타법개정', '별표편집여부': 'N', '공포법령여부': 'Y', '공동부령정보': None, '조문': [{'조문번호': '1', '조문여부': '조문', '조문제목': '목적', '조문시행일자': '20230808', '조문제개정유형': None, '조문이동이전': '000000', '조문이동이후': '000000', '조문변경여부': 'Y', '조문내용': '제1조(목적) 이 법은 10ㆍ27법난과 관련하여 피해를 입은 사람과 불교계의 명예를 회복시켜줌으로써 인권신장과 국민화합에 이바지함을 목적으로 한다. <개정 2023.8.8>\n'}, {'조문번호': '2', '조문여부': '조문', '조문제목': '정의', '조문시행일자': '20230808', '조문제개정유형': None, '조문이동이전': '000000', '조문이동이후': '000000', '조문변경여부': 'Y', '조문내용': '제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2023.8.8>\n', '항': [{'항번호': None, '항제개정유형': None, '항내용': None, '호': [{'호번호': '1.', '호내용': '  1. "10ㆍ27법난"이란 1980년 10월 계엄사령부의 합동수사본부 합동수사단이 불교계 정화를 명분으로 대한불교 조계종의 승려 및 불교 관련자를 강제로 연행ㆍ수사하고, 포고령 위반 수배자 및 불순

In [21]:
import json

for law_text in law_text_list:
    # json 파일로 저장
    with open(f'../../data/law_text/{law_text["법령ID"]}.json', 'w') as f:
        json.dump(law_text, f, ensure_ascii=False, indent=4)

----